

### Generación de Modelos Predictivos con LOGISTIC REGRESSION

Este tipo de modelos requiere una revisión previa de colienalidad.  Tambien permiten obtener información del peso de las variables en el modelo.

In [1]:
tipo_a_usar='CPU'

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score,roc_auc_score, confusion_matrix
from sklearn.ensemble import GradientBoostingClassifier
import xgboost as xgb

Los datos de origen es el mismo sea desde Google Colab o desde el iMac.  La diferencia es que el primero usar referencia a Google Drive y el otro acceso directo desde el sistema operativo

In [3]:
if tipo_a_usar =='GPU':
  from google.colab import drive
  drive.mount('/content/drive')

if tipo_a_usar =='GPU':
  DATOS_LIMPIOS = pd.read_pickle('/content/drive/MyDrive//Innovaciones Tecnológicas Aplicadas/Universidad Autónoma de Occidente/GoogleColab/DATOS3_35.pkl')

if tipo_a_usar == 'CPU':
  DATOS_LIMPIOS = pd.read_pickle('/Users/jaimereinoso/DESARROLLO/MCD/proyectodegrado2/checkPoints/DATOS_LIMPIOS.pkl')

# DATOS_LIMPIOS['LABEL'].value_counts()

Confirmemos qué columnas vienen y cuales no

In [8]:
subcadena = 'LABEL'

columnas_con_subcadena = DATOS_LIMPIOS.columns[DATOS_LIMPIOS.columns.str.contains(subcadena)]

if len(columnas_con_subcadena) > 0:
    print(f"Las siguientes columnas contienen '{subcadena}' como subcadena:")
    for columna in columnas_con_subcadena:
        print(columna)
else:
    print(f"Ninguna columna contiene '{subcadena}' como subcadena en el DataFrame DATOS_LIMPIOS.")

Las siguientes columnas contienen 'LABEL' como subcadena:
LABEL


Tenemos que conseguir cuáles columnas tienen una alta correlación con LABEL.
Por otro lado, tambien queremos eliminar columnas que tienen una muy alta correlación con otras columnas que no sean LABEL.

In [21]:
# Calcular la matriz de correlación
cm = DATOS_LIMPIOS.corr()

# Obtener la correlación de todas las columnas con LABEL
cmlabel = cm['LABEL']

# Obtén el valor absoluto de las correlaciones
cmlabel_abs = cmlabel.abs()

correlation_summary = pd.DataFrame({
    'Correlation': cmlabel,
    'Absolute Correlation': cmlabel_abs
})

# Ordena el DataFrame por valor absoluto en orden descendente
cmlabel = correlation_summary.sort_values(by='Absolute Correlation', ascending=False)

# Muestra el DataFrame ordenado
print(cmlabel)


                                      Correlation  Absolute Correlation
LABEL                                    1.000000              1.000000
prom_nivel_asig_300EIO043               -0.499771              0.499771
prom_nivel_asig_300EIO039               -0.266108              0.266108
prom_nivel_creditos_0                   -0.259072              0.259072
prom_nivel_dpto_EIO                     -0.258825              0.258825
...                                           ...                   ...
prom_nivel_asig_300CSI001                0.000206              0.000206
prom_nivel_hace_10.0                     0.000198              0.000198
prom_nivel_asig_300CSE014                0.000156              0.000156
cursos_per_canc_antes_otros_dpto_IBA     0.000065              0.000065
prom_nivel_dpto_CSI                      0.000017              0.000017

[1954 rows x 2 columns]


ahora obtenemos las columnas con una baja correlación entre columnas que no sean LABEL

In [24]:
cmSinLabel = cm.drop(columns='LABEL', index='LABEL')

columnasFinales = []
umbral = 0.5

for col1 in cmSinLabel.columns:
    correlated = False
    for col2 in cmSinLabel.columns:
        if col1 != col2 and abs(cmSinLabel.loc[col1, col2]) >= umbral:  # Puedes ajustar el umbral según tus necesidades
            correlated = True
            break
    if not correlated:
        columnasFinales.append(col1)

In [25]:
columnasFinales

['estudiante_edad',
 'docente_1',
 'docente_3',
 'docente_4',
 'docente_5',
 'docente_6',
 'docente_8',
 'docente_9',
 'docente_10',
 'docente_11',
 'docente_12',
 'docente_13',
 'docente_15',
 'docente_16',
 'docente_17',
 'docente_18',
 'docente_19',
 'docente_20',
 'docente_22',
 'docente_23',
 'docente_24',
 'docente_25',
 'docente_26',
 'docente_27',
 'docente_28',
 'docente_29',
 'colegio_0040007',
 'colegio_0040102',
 'colegio_0040110',
 'colegio_0040115',
 'colegio_0040464',
 'colegio_0040508',
 'colegio_0040519',
 'colegio_0040584',
 'colegio_0040667',
 'colegio_0040814',
 'colegio_0040899',
 'colegio_0041170',
 'colegio_0041242',
 'colegio_0041245',
 'colegio_0041274',
 'colegio_0041369',
 'colegio_0041465',
 'colegio_0041502',
 'colegio_0041517',
 'colegio_0041588',
 'colegio_0041594',
 'colegio_0041675',
 'colegio_0041767',
 'colegio_0041840',
 'colegio_0041857',
 'colegio_0042122',
 'colegio_0042131',
 'colegio_0042167',
 'colegio_0042169',
 'colegio_0042202',
 'colegio_00

In [8]:
#!pip install imbalanced-learn
from imblearn.over_sampling import SMOTE

# solo tomamos en cuenta los 0 (no desertores) y 1 (desertores)
data = DATOS_LIMPIOS[DATOS_LIMPIOS['LABEL']>=0].copy() 

X = data.drop('LABEL', axis=1)
y = data['LABEL']

# procedemos a hacer el balanceo
smote = SMOTE(sampling_strategy='auto')

# Aplica SMOTE para generar muestras sintéticas
X_resampled, y_resampled = smote.fit_resample(X, y)

Para regresion logistica toca primero eliminar variables altamente correlacionadas.
De hecho, el umbral se vuelve un hiper parámetro a tomar en cuenta para generar varios modelos con distinto umbral.

Se desea dejar varibles altamente correlacionadas con LABEL (positiva o negativamente) y 
eliminar varibles (no LABEL) altamente correlacionadas entre ellas (no LABEL).

In [18]:
dataLR = DATOS_LIMPIOS.copy()
correlation_matrix = dataLR.corr()

# Crear una máscara booleana para las variables altamente correlacionadas
correlation_threshold = 0.7  # Define el umbral de correlación deseado
highly_correlated = (correlation_matrix.abs() >= correlation_threshold) & (correlation_matrix.abs() < 1.0)

# id_docente, grupo ...  no veo a asignatura

In [10]:

# Identificar las variables altamente correlacionadas 
correlated_columns = set()
for col in highly_correlated.columns:
    correlated_columns.update(highly_correlated.index[highly_correlated[col]])

# Eliminar las variables altamente correlacionadas del DataFrame
X_resampled = dataLR.drop(columns=correlated_columns)

habiendo ya eliminado columnas 

In [14]:
# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2)
X_train.shape

(7712, 1118)

In [15]:
# Definir una lista de algoritmos que deseas probar, y si deben usar CPU o GPU
# si usan CPU se pueden correr en el iMac.  Si usan GPU se corren en GOOGLE COLAB 

algorithms = [
#    ('RandomForest',
#         RandomForestClassifier(), {
#             'n_estimators': [50, 100, 200,400],
#             'max_depth': [None, 10, 20, 30],
#             'min_samples_split': [2, 5, 10]
#         },
#         'CPU'
#     ),
    # ('DecisionTree',
    #     DecisionTreeClassifier(), {
    #         'criterion': ['gini', 'entropy'],
    #         'max_depth': [None, 10, 20, 30, 40, 50],
    #         'min_samples_split': [2, 5, 10],
    #         'min_samples_leaf': [1, 2, 4]
    #     },
    #     'CPU'
    # ),
        ('LogisticRegression',
        LogisticRegression(), {
            'penalty': ['l1', 'l2'],           # Tipo de regularización
            'C': [0.001, 0.01, 0.1, 1, 10],   # Parámetro de regularización (inverso de la fuerza de regularización)
            'solver': ['liblinear', 'saga']
        },
        'CPU'
    ),
    # ('SVM', SVC(), {
    #     'C': [0.1],
    #     'kernel': ['linear']
    # },
    # 'CPU'),
    ('XGBoost', xgb.XGBClassifier(), {
        'n_estimators': [50, 100, 200],
        'learning_rate': [0.01, 0.1, 0.2],
        'max_depth': [3, 4, 5],
        'tree_method':['hist']  # no aguantó gpu_hist en google_colab
        },
        'GPU'
    )
]

In [16]:
import joblib
import datetime


for algo_name, model, param_grid, tipo in algorithms:

  if tipo == tipo_a_usar:

    modelo_a_salvar = []

    print('algo:', algo_name, 'model:',model, 'param grid:', param_grid)
    grid_search = GridSearchCV(model, param_grid, cv=5, scoring='roc_auc')
    grid_search.fit(X_train, y_train)

    # Evaluar el modelo
    y_pred = grid_search.best_estimator_.predict(X_test)

    # vamos a salvarlo en modelo_a_salvar
    modelo_a_salvar.append(algo_name)

    accuracy = accuracy_score(y_test, y_pred)
    roc_auc = roc_auc_score(y_test, y_pred)
    conf_matrix = confusion_matrix(y_test, y_pred)

    modelo_a_salvar.append(accuracy)
    modelo_a_salvar.append(roc_auc)
    modelo_a_salvar.append(conf_matrix)
    modelo_a_salvar.append(grid_search.best_estimator_)
    modelo_a_salvar.append(grid_search.best_params_)

    print("Algoritmo:", algo_name)
    print("Accuracy:", accuracy)
    print("Roc_Auc:", roc_auc)
    print("Conf_Matrix:", conf_matrix)

    # Obtén la fecha y hora actual
    now = datetime.datetime.now()
    nombre_archivo = algo_name + now.strftime("%Y-%m-%d_%H-%M-%S") + ".pkl"

    if tipo_a_usar =='GPU':
      from google.colab import files
      joblib.dump(modelo_a_salvar,nombre_archivo)
      files.download(algo_name+'.pkl')
      # queda en download del iMac

    else:
      joblib.dump(modelo_a_salvar,nombre_archivo)
      print('entro aqui')

algo: LogisticRegression model: LogisticRegression() param grid: {'penalty': ['l1', 'l2'], 'C': [0.001, 0.01, 0.1, 1, 10], 'solver': ['liblinear', 'saga']}


/Users/jaimereinoso/DESARROLLO/MCD/proyectodegrado2/venv/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/jaimereinoso/DESARROLLO/MCD/proyectodegrado2/venv/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/jaimereinoso/DESARROLLO/MCD/proyectodegrado2/venv/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/jaimereinoso/DESARROLLO/MCD/proyectodegrado2/venv/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/jaimereinoso/DESARROLLO/MCD/proyectodegrado2/venv/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:3

Algoritmo: LogisticRegression
Accuracy: 0.8661825726141079
Roc_Auc: 0.8660186387100393
Conf_Matrix: [[875 131]
 [127 795]]
entro aqui
